In [19]:
import model as m
import wrangle as w
from importlib import reload
from sklearn.linear_model import LinearRegression

In [20]:
train,validate,test = w.tvt_split(w.wrangle_zillow())

In [21]:
reload(m)
m.select_baseline(train.log_error)

|                 |     SSE |       MSE |     RMSE |
|:----------------|--------:|----------:|---------:|
| Mean Baseline   | 1285.55 | 0.0296687 | 0.172246 |
| Median Baseline | 1289.55 | 0.0297612 | 0.172514 |
### Because mean outperformed median on all metrics,         we will use mean as our baseline

In [ ]:
scaled_train = w.scale(train,['tax_value','calc_sqft','latitude','longitude'])


In [29]:
reload(m)
features = ['latitude','longitude','tax_value','calc_sqft','bed_count','bath_count']
cluster_cols = ['tax_value','calc_sqft','latitude','longitude']
cluster_name = 'cluster_fuck'
clustered_scaled,scaler,clusterer = m.scale_and_cluster(train,features=features,target=train.log_error,cluster_cols=cluster_cols,cluster_name=cluster_name,k=7)
m.generate_regressor(features=clustered_scaled,target=clustered_scaled.log_error,cluster_col=cluster_name,regressor=LinearRegression,normalize=True)

SyntaxError: invalid syntax (model.py, line 2)